1. Downloading dataset from kaggle

In [106]:
!mkdir .kaggle

import json
#In username enter your username obtained from kaggle and key obtained
token = {	"username":"amratye","key":"b7620e5da4833304f5e9ec08b2daba2e"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)
    
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!kaggle config set -n path -v{/content}
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d mmoreaux/audio-cats-and-dogs


mkdir: cannot create directory ‘.kaggle’: File exists
- path is now set to: {/content}
audio-cats-and-dogs.zip: Skipping, found more recently modified local copy (use --force to force download)


In [107]:
!unzip /content/{/content}/datasets/mmoreaux/audio-cats-and-dogs/audio-cats-and-dogs.zip

Archive:  /content/{/content}/datasets/mmoreaux/audio-cats-and-dogs/audio-cats-and-dogs.zip
replace cats_dogs/cat_1.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
  inflating: cats_dogs/test/cats/cat_110.wav  
  inflating: cats_dogs/test/cats/cat_112.wav  
  inflating: cats_dogs/test/cats/cat_115.wav  
  inflating: cats_dogs/test/cats/cat_126.wav  
  inflating: cats_dogs/test/cats/cat_129.wav  
  inflating: cats_dogs/test/cats/cat_130.wav  
  inflating: cats_dogs/test/cats/cat_133.wav  
  inflating: cats_dogs/test/cats/cat_135.wav  
  inflating: cats_dogs/test/cats/cat_137.wav  
  inflating: cats_dogs/test/cats/cat_14.wav  
  inflating: cats_dogs/test/cats/cat_143.wav  
  inflating: cats_dogs/test/cats/cat_144.wav  
  inflating: cats_dogs/test/cats/cat_148.wav  
  inflating: cats_dogs/test/cats/cat_152.wav  
  inflating: cats_dogs/test/cats/cat_158.wav  
  inflating: cats_dogs/test/cats/cat_17.wav  
  inflating: cats_dogs/test/cats/cat_20.wav  
  inflating: cats_dogs/test/cats/cat_24.wav

2. Importing the packages

In [224]:
#importing all libraries
import os
import IPython.display as ipd
import librosa
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Lambda,Input,Dropout,Activation
from tensorflow.keras import backend as K
import numpy as np
from pprint import pprint 

3. Preprocessing the text

In [197]:
def read(path):
  data_dog = [librosa.load(os.path.join(os.path.join(path,'dog'),x))[0] for x in os.listdir(os.path.join(path,'dog'))]
  label_dog = np.zeros_like(data_dog)
  data_cat = [librosa.load(os.path.join(os.path.join(path,'cat'),x))[0] for x in os.listdir(os.path.join(path,'cat'))]
  label_cat = np.ones_like(data_cat)
  return data_dog, data_cat, label_dog, label_cat

In [198]:
def padding(train_x):
  
  max_value = 0
  #finding ther maximum dimensional array for padding other arrays
  for k in train_x:
    temp = np.array(k).shape
    if (max_value <= temp[0]):
      max_value = temp[0]
      
  #padding the other arrays
  for i,f in enumerate(train_x):
    current_shape = np.array(f).shape[0]
    pad_number = max_value - int(current_shape) 
    train_x[i] = list(f)+[0]*pad_number
  return train_x


In [199]:
def preprocess(data_dog,data_cat,label_dog,label_cat):
  
  #it takes time so we defined it under a function after running it one time we can comment out the 11th line as the values of data_dog,data_cat,lable_dog,label_cat are founded
  #0.16 is the maximum value so we are normalizing the dataset
  data = np.concatenate((data_dog,data_cat),axis = 0)/1.16
  data = padding(data)
  label = np.concatenate((label_dog,label_cat),axis = 0)
  train = []

  for i,j in zip(data,label):
  #padding the audio files 
    train.append([i,j])
  '''
  max_arr,min_arr = [],[]
  for i in data:
    max_arr.append(np.max(i))
    min_arr.append(np.min(i))    
  max = np.max(max_arr)
  min = np.min(min_arr)
  print("The minimum and the maximum values are {} and {}".format(max,min))
  '''
  return train

In [200]:
train_path = '/content/cats_dogs/train'
data_dog, data_cat, label_dog, label_cat = read(train_path)
train = np.array(preprocess(data_dog, data_cat, label_dog, label_cat))
np.random.shuffle(train)
train_x,train_y = train[:,0],train[:,1]

In [201]:
test_path = '/content/cats_dogs/test'
data_dog, data_cat, label_dog, label_cat = read(test_path)
test = np.array(preprocess(data_dog, data_cat, label_dog, label_cat)) 
np.random.shuffle(test) 
test_x,test_y = test[:,0],test[:,1]

Building the model

In [248]:
#preparing the base model
def base_model(input_shape):
  model = Sequential()
  model.add(Dense(128,input_shape = input_shape,activation = 'relu'))
  model.add(Dense(128,activation = 'relu'))
  model.add(Dropout(0.1))
  model.add(Dense(128,activation = 'relu'))
  model.add(Dropout(0.3))
  model.add(Dense(128,activation = 'relu'))
  #model.add(Dropout(0.5))
  #model.add(Dense(input_shape,activation = 'relu'))
  return model
  

In [249]:
input_shape = np.array(train_x[0]).shape
input_1 = Input(shape = input_shape)
input_2 = Input(shape = input_shape)

def euclidean(vect):
  x,y = vect
  distance = K.sqrt(K.sum(K.square(x-y),axis = 0))
  return distance

def contrastive_loss(y_true,y_pred):
  margin = 1
  loss = y_true*(K.square(y_pred)) + (1-y_pred)*K.square(K.maximum(margin-y_pred,0))  

sub_model = base_model(input_shape) 
x = sub_model(input_1)
y = sub_model(input_2)
sub_model.summary

layer = Lambda(euclidean)([x,y])
layer = tf.expand_dims(layer,axis = 1)
print(layer.shape)
output = Dense(1,activation = 'sigmoid')(layer)
siamese_model = tf.keras.Model(inputs = [input_1,input_2],outputs = output)


(128, 1)


Compiling the model


In [250]:
siamese_model.compile(optimizer = 'adam',loss = contrastive_loss,metrics = ['accuracy'])

Fitting the model


In [ ]:
train_x = [np.array(x).astype(np.float32) for x in train_x]
print(np.array(train_x).shape)
print(train_y.shape)
siamese_model.fit(np.array(train_x),np.array(train_y).astype(np.float32),validation_split = 0.2,epochs = 20)

In [ ]:
|